# Initialization

Test notebook for the C-MAPPS benchmark. Approach using MLP with Time window. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import numpy as np
import csv
import copy
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

Using TensorFlow backend.


# Single MLP

Use the MLP with 1 hidden layer and 20 Neurons as per reference "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [2]:
def RULmodel_TW(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the model and load the data

Use different time window sizes for the comparisson. "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [3]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]

#Selected as per CNN paper
selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

#Selected from the first section of the paper
windowSize = 1
stride = 1
constRUL = -1

dataFolder = '../CMAPSSData'

#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*windowSize
modelRULSN = RULmodel_TW(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

tModel = TunableModel('ModelRUL_TW', modelRULSN, selected_features, dataFolder, window_size=windowSize,
                      scaler = min_max_scaler, window_stride=stride, constRul = constRUL)
tModel.loadData(verbose=1)
tModel.printData(printTop=True)

Loading data for dataset 1 with window_size of 1, stride of 1 and constRUL of -1. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(20631, 14)
(20631, 1)
Testing data (X, y)
(100, 14)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616  0.45249597 -0.51515152 -0.78048999
  -0.26190476  0.26652452 -0.58823529 -0.60078439 -0.2720277  -0.33333333
   0.42635659  0.44932339]
 [-0.43373494 -0.09396119 -0.29473329  0.25603865 -0.57575758 -0.79951539
  -0.23809524  0.53091684 -0.44117647 -0.674373   -0.17737591 -0.33333333
   0.33333333  0.46202706]
 [-0.31325301 -0.26095487 -0.25894666  0.42028986 -0.45454545 -0.71991385
  -0.5         0.59061834 -0.55882353 -0.65641449 -0.28510966 -0.66666667
   0.25581395  0.24275062]
 [-0.31325301 -0.48768258 -0.33760972  0.48148148 -0.36363636 -0.75096473
  -0.66666667  0.7782516  -0.41176471 -0.6502219  -0.66679492 -0.33333333
   0.14728682  0.32477216]
 [-0.30120482 -0.485

In [20]:
tModel.epochs = 100
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
tModel.trainModel(learningRateScheduler = lrate, verbose=1)

Epoch 1/100
20631/20631 [==============================] - 0s 6us/step - loss: 329.0281 - mean_squared_error: 329.0281
Epoch 2/100
20631/20631 [==============================] - 0s 5us/step - loss: 328.9261 - mean_squared_error: 328.9261
Epoch 3/100
20631/20631 [==============================] - 0s 5us/step - loss: 328.9448 - mean_squared_error: 328.9448
Epoch 4/100
20631/20631 [==============================] - 0s 7us/step - loss: 328.9057 - mean_squared_error: 328.9057
Epoch 5/100
20631/20631 [==============================] - 0s 7us/step - loss: 328.8346 - mean_squared_error: 328.8346
Epoch 6/100
20631/20631 [==============================] - 0s 10us/step - loss: 328.8404 - mean_squared_error: 328.8404
Epoch 7/100
20631/20631 [==============================] - 0s 8us/step - loss: 328.7913 - mean_squared_error: 328.7913
Epoch 8/100
20631/20631 [==============================] - 0s 6us/step - loss: 328.7455 - mean_squared_error: 328.7455
Epoch 9/100
20631/20631 [======================

20631/20631 [==============================] - 0s 8us/step - loss: 327.1133 - mean_squared_error: 327.1133
Epoch 70/100
20631/20631 [==============================] - 0s 7us/step - loss: 327.0547 - mean_squared_error: 327.0547
Epoch 71/100
20631/20631 [==============================] - 0s 6us/step - loss: 327.0926 - mean_squared_error: 327.0926
Epoch 72/100
20631/20631 [==============================] - 0s 6us/step - loss: 327.0334 - mean_squared_error: 327.0334
Epoch 73/100
20631/20631 [==============================] - 0s 5us/step - loss: 327.0193 - mean_squared_error: 327.0193
Epoch 74/100
20631/20631 [==============================] - 0s 5us/step - loss: 326.9543 - mean_squared_error: 326.9543
Epoch 75/100
20631/20631 [==============================] - 0s 4us/step - loss: 326.9869 - mean_squared_error: 326.9869
Epoch 76/100
20631/20631 [==============================] - 0s 6us/step - loss: 326.9559 - mean_squared_error: 326.9559
Epoch 77/100
20631/20631 [===========================

# Evaluate model performance

Evaluate the model performance according to the two scores discussed in the reference. "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [21]:
tModel.evaluateModel(["rhs"], round=2)
print("scores")

#print(tModel.y_pred)

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.trainTime

i = 1
for prediction in tModel.y_pred:
    print('Engine {}, RUL {}, Rectified RUL {}, Real RUL {}'.format(i, prediction, 
                                                                    tModel.y_pred_rectified[i-1], 
                                                                    tModel.y_test[i-1]))
    i = i+1

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 80us/step
scores
Engine 1, RUL [ 120.07231903], Rectified RUL [ 120.], Real RUL [ 112.]
Engine 2, RUL [ 114.58013153], Rectified RUL [ 114.], Real RUL [ 98.]
Engine 3, RUL [ 73.09399414], Rectified RUL [ 73.], Real RUL [ 69.]
Engine 4, RUL [ 92.28057098], Rectified RUL [ 92.], Real RUL [ 82.]
Engine 5, RUL [ 108.06224823], Rectified RUL [ 108.], Real RUL [ 91.]
Engine 6, RUL [ 106.72544861], Rectified RUL [ 106.], Real RUL [ 93.]
Engine 7, RUL [ 103.43912506], Rectified RUL [ 103.], Real RUL [ 91.]
Engine 8, RUL [ 90.87776947], Rectified RUL [ 90.], Real RUL [ 95.]
Engine 9, RUL [ 105.74544525], Rectified RUL [ 105.], Real RUL [ 111.]
Engine 10, RUL [ 103.5271759], Rectified RUL [ 103.], Real RUL [ 96.]
Engine 11, RUL [ 79.55339813], Rectified RUL [ 79.], Real RUL [ 97.]
Engine 12, RUL [ 84.78182983], Rectified RUL [ 84.], Real RUL [ 124.]
Engine 13, RUL [ 90.44035339], Rectified RUL [ 90.], Real RUL [ 95.]
Engine 14, RUL [ 111.1852951], Re

# Load data with rectified labels

Rectify the test set labels according to the constantRUL. "A Time Window Neural Network Based Framework for Remaining Useful Life Estimation"

In [19]:
tModel.constRul = 125
tModel.loadData(verbose=1, rectify_labels = False)
tModel.printData(printTop=True)

Loading data for dataset 1 with window_size of 1, stride of 1 and constRUL of 125. Cros-Validation ratio 0
Data loaded for dataset 1
Printing shapes

Training data (X, y)
(20631, 14)
(20631, 1)
Testing data (X, y)
(100, 14)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616  0.45249597 -0.51515152 -0.78048999
  -0.26190476  0.26652452 -0.58823529 -0.60078439 -0.2720277  -0.33333333
   0.42635659  0.44932339]
 [-0.43373494 -0.09396119 -0.29473329  0.25603865 -0.57575758 -0.79951539
  -0.23809524  0.53091684 -0.44117647 -0.674373   -0.17737591 -0.33333333
   0.33333333  0.46202706]
 [-0.31325301 -0.26095487 -0.25894666  0.42028986 -0.45454545 -0.71991385
  -0.5         0.59061834 -0.55882353 -0.65641449 -0.28510966 -0.66666667
   0.25581395  0.24275062]
 [-0.31325301 -0.48768258 -0.33760972  0.48148148 -0.36363636 -0.75096473
  -0.66666667  0.7782516  -0.41176471 -0.6502219  -0.66679492 -0.33333333
   0.14728682  0.32477216]
 [-0.30120482 -0.48